In [1]:
import geodb.model
from geodb.model import GPSPoint, db_url, GPSTrack

In [2]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [3]:
engine = sqlalchemy.create_engine(db_url(), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
import pandas as pd
import numpy as np

Find longest tracks

In [64]:
elapsed_query = """
select track.*, count(*), min(time) as start, max(time) as end, max(time) - min(time) as elapsed
from track
join point on track_id = track.id
group by track.id
order by elapsed desc
"""
display(pd.read_sql_query(elapsed_query, engine).dropna(subset=['elapsed']))

,id,name,comment,description,source,type,filename,properties,raw,count,start,end,elapsed
8,2503,Aneel Nazareth (9962922),None,Aneel Nazareth (9962922),None,None,inreach.gpx,{},None,23,2019-12-09 22:43:30+00:00,2019-12-11 02:46:30+00:00,1 days 04:03:00
9,2500,Aneel Nazareth (9953683),None,Aneel Nazareth (9953683),None,None,inreach.gpx,{},None,2048,2019-12-07 00:03:15+00:00,2019-12-07 23:40:29+00:00,0 days 23:37:14
10,2497,Aneel Nazareth (9947046),None,Aneel Nazareth (9947046),None,None,inreach.gpx,{},None,2111,2019-12-04 23:35:00+00:00,2019-12-05 21:53:17+00:00,0 days 22:18:17
11,2502,Aneel Nazareth (9959973),None,Aneel Nazareth (9959973),None,None,inreach.gpx,{},None,545,2019-12-09 00:15:00+00:00,2019-12-09 21:23:59+00:00,0 days 21:08:59
12,2501,Aneel Nazareth (9956605),None,Aneel Nazareth (9956605),None,None,inreach.gpx,{},None,1785,2019-12-07 23:40:45+00:00,2019-12-08 14:10:00+00:00,0 days 14:29:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,923,None,None,None,None,car,2019-11-28.gpx,{},None,1,2019-11-28 01:51:46+00:00,2019-11-28 01:51:46+00:00,0 days 00:00:00
2500,1867,None,None,None,None,car,2019-12-04.gpx,{},None,1,2019-12-05 02:27:10+00:00,2019-12-05 02:27:10+00:00,0 days 00:00:00
2501,1014,None,None,None,None,car,2019-11-28.gpx,{},None,1,2019-11-28 08:08:10+00:00,2019-11-28 08:08:10+00:00,0 days 00:00:00
2502,520,柚之茶屋,None,None,None,Waypoint,None,None,None,1,2019-11-27 05:12:36+00:00,2019-11-27 05:12:36+00:00,0 days 00:00:00


Pick a track

In [65]:
track2503 = session.query(GPSTrack).get(2503)

In [66]:
track2503.name

'Aneel Nazareth (9962922)'

Find all overlapping tracks

In [73]:
overlapping_query = """
select * from track where id in
(select distinct(track_id) from point
where point.time between (select min(time) from point where track_id = 2503) 
                     and (select max(time) from point where track_id = 2503))
"""
display(pd.read_sql_query(overlapping_query, engine))

,id,name,comment,description,source,type,filename,properties,raw
0,2439,Keisei Ueno Station (KS01) (京成上野駅),None,None,None,Waypoint,None,None,None
1,2440,Keisei Narita Airport Terminal 2-3 Station (KS...,None,None,None,Waypoint,None,None,None
2,2441,Terminal 2 (第2ターミナル),None,None,None,Waypoint,None,None,None
3,2442,ASIAN CAFE Bowl Bowl,None,None,None,Waypoint,None,None,None
4,2443,NRT - GATE 74 (Terminal 2),None,None,None,Waypoint,None,None,None
...,...,...,...,...,...,...,...,...,...
93,2636,Keisei Ueno Station (KS01) (京成上野駅),None,None,None,Waypoint,None,None,None
94,2637,Keisei Narita Airport Terminal 2-3 Station (KS...,None,None,None,Waypoint,None,None,None
95,2638,Terminal 2 (第2ターミナル),None,None,None,Waypoint,None,None,None
96,2639,ASIAN CAFE Bowl Bowl,None,None,None,Waypoint,None,None,None


In [68]:
overlapping = [session.query(GPSTrack).get(int(t)) for t in pd.read_sql_query(overlapping_query, engine)[['id']].values]

In [69]:
len(overlapping)

98

In [74]:
query = """
select track.*, count(*), min(time) as start, max(time) as end, max(time) - min(time) as elapsed
from track
join point on track.id = point.track_id
where point.time between (select min(p2.time) from point p2 where track_id = 2503)
                     and (select max(p3.time) from point p3 where track_id = 2503)
group by track.id
order by elapsed desc
"""
display(pd.read_sql_query(query, engine))

,id,name,comment,description,source,type,filename,properties,raw,count,start,end,elapsed
0,2503,Aneel Nazareth (9962922),None,Aneel Nazareth (9962922),None,None,inreach.gpx,{},None,23,2019-12-09 22:43:30+00:00,2019-12-11 02:46:30+00:00,1 days 04:03:00
1,2509,20191210,None,None,None,None,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None,400,2019-12-10 06:01:17+00:00,2019-12-10 13:07:36+00:00,0 days 07:06:19
2,2508,AAL176-20191210,None,RJAA - KDFW,None,None,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None,111,2019-12-10 02:37:43+00:00,2019-12-10 05:59:52+00:00,0 days 03:22:09
3,2544,None,None,None,None,airplane,2019-12-10-edited.gpx,{},None,27,2019-12-10 16:30:06+00:00,2019-12-10 18:22:01+00:00,0 days 01:51:55
4,2592,None,None,None,None,train,2019-12-09-edited.gpx,{},None,342,2019-12-09 23:00:09+00:00,2019-12-09 23:46:32+00:00,0 days 00:46:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,2481,Austin Bergstrom International Airport (AUS),None,None,None,Waypoint,None,None,None,1,2019-12-10 18:22:16+00:00,2019-12-10 18:22:16+00:00,0 days 00:00:00
94,2604,None,None,None,None,airplane,2019-12-09-edited.gpx,{},None,1,2019-12-10 00:25:32+00:00,2019-12-10 00:25:32+00:00,0 days 00:00:00
95,2480,Gate C35,None,None,None,Waypoint,None,None,None,1,2019-12-10 15:01:42+00:00,2019-12-10 15:01:42+00:00,0 days 00:00:00
96,2479,Gate C35,None,None,None,Waypoint,None,None,None,1,2019-12-10 14:29:44+00:00,2019-12-10 14:29:44+00:00,0 days 00:00:00


In [75]:
from ipyleaflet import (
    Map,
    Marker,
    Polyline, 
    Popup,
    GeoJSON,
    DrawControl
)
from ipywidgets import HTML

from traitlets import link

In [76]:
len(overlapping[0].points)

1

In [77]:
ps = [[float(p.coords[1]), float(p.coords[0])] for p in track2503.points]
bounds = [list(ps[0]), list(ps[0])]
for p in ps:
    if bounds[0][0] > p[0]:
        bounds[0][0] = p[0]
    if bounds[0][1] > p[1]:
        bounds[0][1] = p[1]
    if bounds[1][0] < p[0]:
        bounds[1][0] = p[0]
    if bounds[1][1] < p[1]:
        bounds[1][1] = p[1]
bounds

[[30.20118, -97.713281], [35.783671, 140.38918]]

In [78]:
center = [(bounds[0][0] + bounds[1][0])/2, (bounds[0][1] + bounds[1][1])/2 - 180]

In [79]:
center

[32.992425499999996, -158.6620505]

In [80]:
def add_track(m, t):
    l = track.as_polyline(
        fill=False,
        weight=1)
    m.add_layer(l)
    l.popup = HTML(f"{track.id}: {track.name}")

In [81]:
m = Map(center=center, zoom=2, close_popup_on_click=False)
for track in overlapping:
    add_track(m, track)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [84]:
def clone_model(model, **kwargs):
    """Clone an arbitrary sqlalchemy model object without its primary key values."""
    # Ensure the model’s data is loaded before copying.
    model.id

    table = model.__table__
    non_pk_columns = [k for k in table.columns.keys() if k not in table.primary_key]
    data = {c: getattr(model, c) for c in non_pk_columns}
    data.update(kwargs)

    clone = model.__class__(**data)
    return clone

In [169]:
query = """
select type, count(*) from track
join point on track.id = track_id
group by type
"""
display(pd.read_sql_query(query, engine))

,type,count
0,cycling,79
1,airplane,3061
2,car,11959
3,None,29110
4,stationary,2790
5,motorcycle,919
6,tram,194
7,train,8794
8,walking,31200
9,metro,1227


In [180]:
overlapping_points_query = """
select min(point.id) as id, latitude, longitude, time from point
join track on track.id = track_id
where point.time between (select min(time) from point where track_id = 2503) 
                     and (select max(time) from point where track_id = 2503)
and (track.type is null or track.type != 'Waypoint')
group by latitude, longitude, time
order by time
"""
display(pd.read_sql_query(overlapping_points_query, engine))

,id,latitude,longitude,time
0,89057,35.710963,139.778258,2019-12-09 22:43:30+00:00
1,91470,35.711317,139.777343,2019-12-09 22:44:43+00:00
2,91471,35.711175,139.777160,2019-12-09 22:44:50+00:00
3,91472,35.711037,139.776997,2019-12-09 22:44:56+00:00
4,91473,35.711046,139.776981,2019-12-09 22:44:58+00:00
...,...,...,...,...
2141,89166,32.867430,-97.054870,2019-12-10 20:02:14+00:00
2142,89167,32.878540,-97.054850,2019-12-10 20:02:30+00:00
2143,89168,32.889270,-97.054760,2019-12-10 20:02:46+00:00
2144,89078,30.299531,-97.713281,2019-12-10 23:41:00+00:00


In [201]:
overlapping_points = [session.query(GPSPoint).get(int(t)) for t in pd.read_sql_query(overlapping_points_query, engine)[['id']].values]

In [202]:
len(overlapping_points)

2130

In [203]:
merged = GPSTrack(name="merged")

In [204]:
merged.points = [clone_model(p, track=merged) for p in overlapping_points]

In [205]:
m2 = Map(center=(0., 0.), zoom=2, close_popup_on_click=False)
m2.add_layer(merged.as_polyline(
        fill=False,
        weight=1))

m2

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [206]:
session.add(merged)

In [207]:
session.commit()

In [212]:
query = """
select count(point.id), latitude, longitude, time from point
group by latitude, longitude, time
order by 1 desc
"""
display(pd.read_sql_query(query, engine))

,count,latitude,longitude,time
0,8,32.894658,-97.037454,2019-12-10 17:01:15+00:00
1,8,35.711024,139.774378,2019-12-09 22:51:55+00:00
2,7,35.748532,140.389507,2019-12-10 02:30:41+00:00
3,7,35.744097,140.391497,2019-12-10 02:36:05+00:00
4,7,35.750816,140.387975,2019-12-10 02:29:54+00:00
...,...,...,...,...
78610,1,34.694475,134.907278,2019-11-26 01:37:18+00:00
78611,1,34.694607,135.500243,2019-11-25 07:56:51+00:00
78612,1,34.694632,137.579504,2019-11-25 05:54:46+00:00
78613,1,34.694636,137.575319,2019-11-25 05:54:52+00:00


In [260]:
query = """
select t1i, t2i, t1n, t2n, t1f, t2f, count(*) from (
select p1.time, p1.id, p1.latitude, p1.longitude,
p2.id, p2.latitude, p2.longitude,
p1.track_id as t1i, p2.track_id as t2i, t1.name t1n, t2.name t2n,
t1.filename t1f, t2.filename t2f
from point p1
join point p2 on p1.id < p2.id and p1.time = p2.time
join track t1 on p1.track_id = t1.id
join track t2 on p2.track_id = t2.id
where p1.latitude = p2.latitude
and p1.longitude = p2.longitude
and t2.name != 'merged'
) x group by t1i, t2i, t1n, t2n, t1f, t2f
order by count(*) desc
"""
display(pd.read_sql_query(query, engine))

,t1i,t2i,t1n,t2n,t1f,t2f,count
0,2484,2486,Aneel Nazareth (9912559),Aneel Nazareth (9890112),inreach.gpx,inreach.gpx,4
1,2503,2503,Aneel Nazareth (9962922),Aneel Nazareth (9962922),inreach.gpx,inreach.gpx,2
2,35,81,Home,Home,None,None,1
3,36,82,Shag Salon,Shag Salon,None,None,1
4,37,83,Home,Home,None,None,1
...,...,...,...,...,...,...,...
94,2490,2490,Aneel Nazareth (9926387),Aneel Nazareth (9926387),inreach.gpx,inreach.gpx,1
95,2492,2492,Aneel Nazareth (9931878),Aneel Nazareth (9931878),inreach.gpx,inreach.gpx,1
96,2501,2501,Aneel Nazareth (9956605),Aneel Nazareth (9956605),inreach.gpx,inreach.gpx,1
97,2478,2548,Terminal D,Terminal D,None,None,1


In [234]:
query = """
select * from track where id = 1 or id = 47
"""
display(pd.read_sql_query(query, engine))

,id,name,comment,description,source,type,filename,properties,raw
0,1,None,None,None,None,car,2019-11-23 2.gpx,{},None
1,47,None,None,None,None,car,2019-11-23.gpx,{},None


In [239]:
query = """
select * from point p1
full join point p2 on p1.time = p2.time
where p1.track_id = 1
and p2.track_id = 47
"""
display(pd.read_sql_query(query, engine))

,id,latitude,longitude,elevation,time,tz,properties,track_id,id,latitude,longitude,elevation,time,tz,properties,track_id
0,1,30.299837,-97.713184,182.077484,2019-11-22 15:33:13+00:00,None,{},1,1416,30.299837,-97.713184,182.077484,2019-11-22 15:33:13+00:00,None,{},47
1,2,30.300902,-97.713740,185.102401,2019-11-22 15:34:15+00:00,None,{},1,1417,30.300902,-97.713740,185.102401,2019-11-22 15:34:15+00:00,None,{},47
2,3,30.300977,-97.713790,185.318987,2019-11-22 15:34:21+00:00,None,{},1,1418,30.300977,-97.713790,185.318987,2019-11-22 15:34:21+00:00,None,{},47
3,4,30.300995,-97.713804,185.403060,2019-11-22 15:34:26+00:00,None,{},1,1419,30.300995,-97.713804,185.403060,2019-11-22 15:34:26+00:00,None,{},47
4,5,30.301005,-97.713813,185.465247,2019-11-22 15:34:30+00:00,None,{},1,1420,30.301005,-97.713813,185.465247,2019-11-22 15:34:30+00:00,None,{},47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,164,30.300871,-97.721753,187.115229,2019-11-22 15:51:49+00:00,None,{},1,1579,30.300871,-97.721753,187.115229,2019-11-22 15:51:49+00:00,None,{},47
164,165,30.300547,-97.721661,185.702939,2019-11-22 15:51:56+00:00,None,{},1,1580,30.300547,-97.721661,185.702939,2019-11-22 15:51:56+00:00,None,{},47
165,166,30.300402,-97.721590,185.265332,2019-11-22 15:52:01+00:00,None,{},1,1581,30.300402,-97.721590,185.265332,2019-11-22 15:52:01+00:00,None,{},47
166,167,30.300322,-97.721554,185.496381,2019-11-22 15:52:06+00:00,None,{},1,1582,30.300322,-97.721554,185.496381,2019-11-22 15:52:06+00:00,None,{},47


In [241]:
query = """
select count(*) from point where track_id = 47
"""
display(pd.read_sql_query(query, engine))

,count
0,168


In [246]:
def tracks_identical(session, t1, t2):
        track1 = session.query(GPSTrack).get(t1)
        track2 = session.query(GPSTrack).get(t2)
        return [(p.time, p.latitude, p.longitude, p.elevation) for p in track1.points] == \
    [(p.time, p.latitude, p.longitude, p.elevation) for p in track2.points]

In [258]:
tracks_identical(session, 2504, 2505)

True

In [259]:
session.delete(session.query(GPSTrack).get(2505))
session.commit()

In [261]:
query = """
select p1.time, p1.id, p1.latitude, p1.longitude,
p2.id, p2.latitude, p2.longitude,
p1.track_id as t1i, p2.track_id as t2i, t1.name t1n, t2.name t2n,
t1.filename t1f, t2.filename t2f
from point p1
join point p2 on p1.id < p2.id and p1.time = p2.time
join track t1 on p1.track_id = t1.id
join track t2 on p2.track_id = t2.id
where p1.latitude = p2.latitude
and p1.longitude = p2.longitude
and t2.name != 'merged'
"""
display(pd.read_sql_query(query, engine))

,time,id,latitude,longitude,id,latitude,longitude,t1i,t2i,t1n,t2n,t1f,t2f
0,2019-11-20 01:59:12+00:00,1402,30.299755,-97.713141,2817,30.299755,-97.713141,33,79,Home,Home,None,None
1,2019-11-22 15:52:14+00:00,1403,30.300205,-97.721437,2818,30.300205,-97.721437,34,80,Nails Of America,Nails Of America,None,None
2,2019-11-22 16:55:06+00:00,1404,30.299717,-97.713213,2819,30.299717,-97.713213,35,81,Home,Home,None,None
3,2019-11-22 18:33:05+00:00,1405,30.247303,-97.773265,2820,30.247303,-97.773265,36,82,Shag Salon,Shag Salon,None,None
4,2019-11-22 19:22:28+00:00,1406,30.299823,-97.713028,2821,30.299823,-97.713028,37,83,Home,Home,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2019-11-26 02:10:45+00:00,66892,34.835608,134.692555,66893,34.835608,134.692555,2488,2488,Aneel Nazareth (9919077),Aneel Nazareth (9919077),inreach.gpx,inreach.gpx
99,2019-11-28 06:39:46+00:00,70698,33.812905,135.773544,70699,33.812905,135.773544,2490,2490,Aneel Nazareth (9926387),Aneel Nazareth (9926387),inreach.gpx,inreach.gpx
100,2019-12-08 12:34:39+00:00,88489,35.712422,139.777253,88490,35.712422,139.777253,2501,2501,Aneel Nazareth (9956605),Aneel Nazareth (9956605),inreach.gpx,inreach.gpx
101,2019-12-09 23:03:30+00:00,89058,35.727153,139.772701,89059,35.727153,139.772701,2503,2503,Aneel Nazareth (9962922),Aneel Nazareth (9962922),inreach.gpx,inreach.gpx


In [296]:
redundant_waypoint_query = """
select name, time, count(*), max(track.id) as id from track
join point on track.id = track_id
where type = 'Waypoint'
group by 1, 2
having count(*) > 1
order by 3 desc
"""
display(pd.read_sql_query(redundant_waypoint_query, engine))

,name,time,count,id


In [293]:
redundant = [session.query(GPSTrack).get(int(p)) for p in pd.read_sql_query(redundant_waypoint_query, engine)[['id']].values]
len(redundant)

2

In [294]:
for r in redundant:
    if r is not None:
        session.delete(r)
session.commit()